<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/MixMatch_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-9a3be293-1831-3bf7-0dc1-25b3068f33b9)


In [ ]:
!git clone -b development https://github.com/jmarrietar/NST_MixMatch_HAM10000.git

Cloning into 'NST_MixMatch_HAM10000'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 90 (delta 27), reused 64 (delta 16), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [ ]:
!pip install progress

  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8075 sha256=6c8539efed56a160e7f5ea4e94dec4daf6f56cc81db6f418f26aae446e6de95d
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 13.5MB/s 


In [ ]:
cd /content/NST_MixMatch_HAM10000/

/content/NST_MixMatch_HAM10000


### Download Data 

In [ ]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [ ]:
LABELED = 'sample@4000_images'
URL_LABELED = data_samples[LABELED]
UNLABELED = 'sample@20000'
URL_UNLABELED = data_samples[UNLABELED]

In [ ]:
!gdown $URL_LABELED

Downloading...
From: https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2
To: /content/NST_MixMatch_HAM10000/sample@4000_images.zip
206MB [00:01, 110MB/s]


In [ ]:
import zipfile
local_zip = '{}.zip'.format(LABELED)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/NST_MixMatch_HAM10000')
zip_ref.close()

In [ ]:
!gdown $URL_UNLABELED

Downloading...
From: https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO
To: /content/NST_MixMatch_HAM10000/sample@20000.zip
1.08GB [00:10, 106MB/s] 


In [ ]:
import zipfile
local_zip = '{}.zip'.format(UNLABELED)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/NST_MixMatch_HAM10000')
zip_ref.close()

In [ ]:
VAL = 'voets_test_images'
URL_VAL = data_samples[VAL]

In [ ]:
!gdown $URL_VAL

Downloading...
From: https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd
To: /content/NST_MixMatch_HAM10000/voets_test_images.zip
475MB [00:04, 99.5MB/s]


In [ ]:
import zipfile
local_zip = '{}.zip'.format(VAL)
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/NST_MixMatch_HAM10000')
zip_ref.close()

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
auth.authenticate_user()

In [ ]:
root_path = 'drive/My Drive/Project'

### Code

In [ ]:
from __future__ import print_function, division
import random
import argparse
import numpy as np
import os
import zipfile
import torch
from torch.utils.data import DataLoader
from torchvision import models
from datetime import datetime

In [ ]:
from pytorch_src.data import *
from pytorch_src.batch import *
from pytorch_src.main import create_model, WeightEMA

#### Arguments

In [ ]:
parser = argparse.ArgumentParser()

parser.add_argument('--null_space_tuning', action='store_true',
                    help='Turn on null space training')
parser.add_argument('--alpha', type=float, default=10,
                    help='Determines the impact of null space tuning')
parser.add_argument('--mixup_alpha', type=float, default=0.75,
                    help='Determines contribution of paired image in MixUp')
parser.add_argument('--lambda_u', type=float, default=25.0,
                    help='Determines contribution of MixMatch Loss')
parser.add_argument('--T', type=float, default=0.5,
                    help='Determines sharpening temperature for entropy minimization while guessing labels for unlabeled data')
parser.add_argument('--ema-decay', type=float, default=0.999,
                    help='Decay for exponential moving average')
parser.add_argument('--mixmatch', action='store_true',
                    help='Turn on MixMatch')
parser.add_argument('--labeled_train_file', type=str, default='train_labeled.txt',
                    help='This file should contain the path to an image as well and an integer specifying its class (comma separated) on each line')
parser.add_argument('--unlabeled_train_file', type=str, default='train_unlabeled.txt',
                    help='This file should contain the path to an image as well and an integer specifying its class (comma separated) on each line')
parser.add_argument('--val_file', type=str, default='val.txt',
                    help='This file should contain the path to an image as well and an integer specifying its class (comma separated) on each line')
parser.add_argument('--test_file', type=str, default='test.txt',
                    help='This file should contain the path to an image as well and an integer specifying its class (comma separated) on each line')
parser.add_argument('--out_dir', type=str, default='out/',
                    help='Path to output directory')
parser.add_argument('--mode', type=str, default='train',
                    help='Determines whether to backpropagate or not')
parser.add_argument('--batch_size', type=int, default=6,
                    help='Decides size of each training batch')
parser.add_argument('--lr', type=float, default=0.0001,
                    help='Optimizer\'s learning rate')
parser.add_argument('--total_epochs', type=int, default=75,
                    help='Maximum number of epochs for training')
parser.add_argument('--early_stop', type=int, default=-1,
                    help='Decide to stop early after this many epochs in which the validation loss increases (-1 means no early stopping)')
parser.add_argument('--decay_epoch', type=int, default=-1,
                    help='Decay the learning rate after every this many epochs (-1 means no lr decay)')
parser.add_argument('--decay_rate', type=float, default=0.10,
                    help='Rate at which the learning rate will be decayed')
parser.add_argument('--continue_training', action='store_true',
                    help='Continue training from saved model')
parser.add_argument('--rampup', default=75, type=int, metavar='N',
                    help='number of epochs before unlabeled terms reach full value')

_StoreAction(option_strings=['--rampup'], dest='rampup', nargs=None, const=None, default=75, type=<class 'int'>, choices=None, help='number of epochs before unlabeled terms reach full value', metavar='N')

In [ ]:
config = parser.parse_args(['--alpha', '10',
                   '--mixup_alpha', '0.75',
                   '--lambda_u', '0.1',
                   '--T', '0.5',
                   '--ema-decay', '0.999',
                   '--mixmatch',
                   '--labeled_train_file', 'train_labeled.csv',
                   '--unlabeled_train_file', 'train_unlabeled.csv',
                   '--val_file', 'val.csv',
                   '--test_file', 'test.csv',
                   '--out_dir', 'out',
                   '--mode', 'train',
                   '--batch_size', '6',
                   '--lr', '1e-5',
                   '--total_epochs', '100',
                   '--early_stop', '50',
                   '--decay_epoch', '-1',
                   '--decay_rate', '1e-3',
                   '--rampup', '75'])

In [ ]:
print(config)
os.makedirs(config.out_dir, exist_ok=True)

Namespace(T=0.5, alpha=10.0, batch_size=6, continue_training=False, decay_epoch=-1, decay_rate=0.001, early_stop=50, ema_decay=0.999, labeled_train_file='train_labeled.csv', lambda_u=0.1, lr=1e-05, mixmatch=True, mixup_alpha=0.75, mode='train', null_space_tuning=False, out_dir='out', rampup=75, test_file='test.csv', total_epochs=100, unlabeled_train_file='train_unlabeled.csv', val_file='val.csv')


In [ ]:
os.makedirs(config.out_dir, exist_ok=True)

#### Train Model

In [ ]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:
train_dataset = Train_Dataset(config.labeled_train_file, config.unlabeled_train_file, config)
val_dataset = Test_Dataset(config.val_file, config)

Loading images: 100%|██████████| 8790/8790 [00:29<00:00, 294.43it/s]


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=False, **kwargs)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False, **kwargs)

In [ ]:
model = create_model(device)
ema_model = create_model(device, ema=True)

Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
ema_optimizer= WeightEMA(model, ema_model, alpha=config.ema_decay, lr=config.lr)

In [ ]:
train_loss_file = '{}/train_loss.txt'.format(config.out_dir)
f = open(train_loss_file, 'w')
f.close()
validate_loss_file = '{}/validate_loss.txt'.format(config.out_dir)
f = open(validate_loss_file, 'w')
f.close()
train_accuracy_file = '{}/train_accuracy.txt'.format(config.out_dir)
f = open(train_accuracy_file, 'w')
f.close()
validate_accuracy_file = '{}/validate_accuracy.txt'.format(config.out_dir)
f = open(validate_accuracy_file, 'w')
f.close()

In [ ]:
#model_file = '{}/model'.format(config.out_dir)
model_file = '../{}/{}/{}_{}'.format(root_path, "model",  "MixMatchModel", LABELED)

In [ ]:
model_file

'../drive/My Drive/Project/model/MixMatchModel_sample@4000_images'

In [ ]:
# Continue previous Training 
if config.continue_training:
    model.load_state_dict(torch.load(model_file))
    model = model.to(device)

In [ ]:
seq_increase = 0
min_loss = 10000

In [ ]:
for epoch in range(1, config.total_epochs+1):
    loss = train(model, device, train_loader, optimizer, config, ema_optimizer, epoch)

    with open(train_loss_file, "a") as file:
        file.write(str(loss))
        file.write('\n')

    #loss, accuracy, confusion = test(ema_model, device, val_loader)
    loss, accuracy, confusion = test(model, device, val_loader)

    class_accuracies = '\t\tAccuracy by class: '
    for i in range(len(confusion)):
        correct = confusion[i,i]
        total = sum(confusion[:,i])
        class_accuracies += '{0}--{1:.2f}%   '.format(i, (correct/total)*100)

    print(class_accuracies)

    if epoch % 5  ==0 and (config.mixmatch or config.null_space_tuning) and epoch != config.total_epochs:
        train_dataset.assign_class(model, device)
        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=False, **kwargs)


    with open(validate_loss_file, "a") as file:
        file.write(str(loss))
        file.write('\n')
    with open(validate_accuracy_file, "a") as file:
        file.write(str(accuracy))
        file.write('\n')

    if loss < min_loss:
        min_loss = loss
        with open(model_file, 'wb') as f:
            #torch.save(ema_model.state_dict(), f)
            torch.save(model.state_dict(), f)

    elif config.early_stop != -1:
        if loss > min_loss:
            seq_increase += 1
            if seq_increase == config.early_stop:
                break
        else:
            seq_increase = 0

    if config.decay_epoch != -1:
        if epoch % config.decay_epoch == 0:
            config.lr = config.lr * config.decay_rate
            for param_group in optimizer.param_groups:
                param_group['lr'] = config.lr

       	Avg Loss: 0.6384  Accuracy: 76.11%: 100%|██████████| 1465/1465 [01:54<00:00, 12.75it/s]


		Accuracy by class: 0--63.17%   1--89.05%   


       	Avg Loss: 0.6201  Accuracy: 78.77%: 100%|██████████| 1465/1465 [01:52<00:00, 13.02it/s]


		Accuracy by class: 0--65.18%   1--92.36%   


  0%|          | 0/167 [00:00<?, ?it/s]

...

  0%|          | 0/167 [00:00<?, ?it/s]

		Accuracy by class: 0--78.87%   1--88.62%   


Reassigning unlabeled image labels:   0%|          | 3/20000 [00:00<16:12, 20.57it/s]

		Accuracy by class: 0--81.40%   1--90.49%   


Reassigning unlabeled image labels:  97%|█████████▋| 19379/20000 [15:35<00:28, 21.75it/s]

In [ ]:
1+1

2

In [ ]:
model_file

'../drive/My Drive/Project/model/MixMatchModel_sample@2000'

In [ ]:
with open(model_file, 'wb') as f:
    torch.save(model.state_dict(), f)

# Debug

In [ ]:
import torch
import numpy as np
import zipfile
from skimage import io, transform
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import models
from torchvision import transforms

In [ ]:
!gdown https://drive.google.com/uc?id=1HaUAxDtN4BNj0hpH8QYGmiX39Va-Ke8p

Downloading...
From: https://drive.google.com/uc?id=1HaUAxDtN4BNj0hpH8QYGmiX39Va-Ke8p
To: /content/NST_MixMatch_HAM10000/messidor2.zip
106MB [00:02, 39.9MB/s] 


In [ ]:
local_zip = '{}.zip'.format("messidor2")
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/NST_MixMatch_HAM10000/')
zip_ref.close()

In [ ]:
class Test_Dataset(Dataset):
    def __init__(self, fname):
        self.data = {}
        with open(fname, 'r') as f:
            for l in f.readlines():
                l = l.split(',')
                self.data[l[0]] = int(l[1])
        self.keys = list(self.data.keys())

        with tqdm(total=len(self.keys)) as pbar:
            for key in self.keys:
                self.data[key] = (self.data[key], self.load(key))
                pbar.set_description("Loading images")
                pbar.update(1)

        self.norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.keys)

    def load(self, fname):
        img = io.imread(fname)
        return img

    def preprocess(self, img):
        img = img / 255.0

        img = transform.resize(img, [224, 224, 3], mode='constant', anti_aliasing=True)
        img = self.tensor(img)
        img = self.norm(img.float())

        img = img.type(torch.float32)
        return img

    def __getitem__(self, idx):
        key = self.keys[idx]
        label = self.data[key][0]
        img = self.data[key][1]

        img = self.preprocess(img)

        return {'image': img, \
                'target': torch.tensor(label), \
                'file': key}

In [ ]:
# Make predictions on  Kaggle Test
val_dataset = Test_Dataset('test.csv')

Loading images: 100%|██████████| 1748/1748 [00:07<00:00, 237.17it/s]


In [ ]:
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
loader = val_loader

In [ ]:
y_true = []
y_pred = []

In [ ]:
model.eval()
with torch.no_grad():
    with tqdm(total=len(loader)) as pbar:
        for batch_idx, sample in enumerate(loader):
            data = sample['image']
            target = sample['target']
            file = sample['file']

            labels = target.to(device)
            target = torch.zeros(target.shape[0], 2).scatter_(1, target.view(-1, 1), 1)
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = output.to(device)

            # Yo le agrege esto
            y_true.append(labels.cpu().detach().numpy()[0])
            y_pred.append(np.argmax(output.cpu().detach().numpy()))
            
            pbar.update(1)

100%|██████████| 1748/1748 [01:27<00:00, 19.94it/s]


In [ ]:
# Alternative way to calculate AUC without y_pred_prob
import numpy as np
from sklearn import metrics
#y = np.array([0, 0, 1, 1])
#pred = np.array([0, 0, 1, 0])
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=1)
metrics.auc(fpr, tpr)

0.7073830409356726